# Day 1: 초기 데이터 탐색 (Initial Data Exploration) - v2.0

**날짜**: 2025-07-04

**목표**: 
- 데이터 소스 확인 및 다운로드
- 각 데이터셋의 기본 구조 파악
- 자치구명 필드 표기 확인

**개선 사항** (v2.0):
- ✅ 공통 함수를 utils 모듈로 분리
- ✅ 매직 넘버를 constants.py로 분리
- ✅ OS별 한글 폰트 자동 설정
- ✅ 코드 중복 제거 및 함수화
- ✅ 에러 핸들링 추가

In [ ]:
# 필요한 라이브러리 임포트
import sys
import os

# utils 모듈 경로 추가
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 커스텀 모듈 임포트
from utils.constants import (
    SEOUL_DISTRICTS, CCTV_RANGE, CRIME_RANGE, CCTV_EFFECT_CRIMES,
    POPULATION_CONFIG, RANDOM_SEED, DATA_PATHS
)
from utils.helpers import (
    set_korean_font, set_plot_style, print_data_info, save_csv_safely,
    validate_data, plot_distribution, create_summary_stats,
    plot_multiple_distributions, check_district_consistency
)

warnings.filterwarnings('ignore')

# 한글 폰트 및 시각화 스타일 설정
set_korean_font()
set_plot_style()

# 출력 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

print("\n✅ 모든 라이브러리 및 모듈 임포트 완료")
print(f"📊 분석 대상: 서울시 {len(SEOUL_DISTRICTS)}개 자치구")
print(f"🎲 랜덤 시드: {RANDOM_SEED}")

## 1. CCTV 설치 현황 데이터 생성 및 탐색

In [ ]:
# 샘플 CCTV 데이터 생성 (실제 환경에서는 다운로드한 파일 로드)
np.random.seed(RANDOM_SEED)

cctv_data = {'자치구': SEOUL_DISTRICTS}

# CCTV 유형별 대수 생성 (constants에서 정의한 범위 사용)
for cctv_type, (min_val, max_val) in CCTV_RANGE.items():
    cctv_data[cctv_type] = np.random.randint(min_val, max_val, size=len(SEOUL_DISTRICTS))

cctv_df = pd.DataFrame(cctv_data)

# 총 CCTV 대수 계산 (행 단위 합계)
cctv_df['총_CCTV'] = cctv_df[list(CCTV_RANGE.keys())].sum(axis=1)

# CSV 파일로 저장
save_csv_safely(
    cctv_df,
    os.path.join(DATA_PATHS['raw'], 'cctv_seoul_2023_sample.csv'),
    index=False,
    encoding='utf-8-sig'
)

# 데이터 정보 출력
print_data_info(cctv_df, "CCTV")

In [ ]:
# CCTV 데이터 검증
validate_data(
    cctv_df,
    expected_rows=25,
    required_columns=['자치구', '방범용', '총_CCTV']
)

# 기초 통계
print("\n📊 CCTV 데이터 기초 통계:")
summary_stats = create_summary_stats(cctv_df, list(CCTV_RANGE.keys()) + ['총_CCTV'])
summary_stats

## 2. 범죄 통계 데이터 생성 및 탐색

In [ ]:
# 샘플 범죄 데이터 생성
np.random.seed(RANDOM_SEED)

crime_data = {'자치구': SEOUL_DISTRICTS}

# 범죄 유형별 건수 생성
for crime_type, (min_val, max_val) in CRIME_RANGE.items():
    crime_data[crime_type] = np.random.randint(min_val, max_val, size=len(SEOUL_DISTRICTS))

crime_df = pd.DataFrame(crime_data)

# CCTV 효과 범죄 합계 (절도 + 강도 + 차량범죄)
crime_df['CCTV효과범죄_합계'] = crime_df[CCTV_EFFECT_CRIMES].sum(axis=1)

# 총 범죄 건수
crime_df['총_범죄'] = crime_df[list(CRIME_RANGE.keys())].sum(axis=1)

# 저장
save_csv_safely(
    crime_df,
    os.path.join(DATA_PATHS['raw'], 'crime_seoul_2023_sample.csv'),
    index=False,
    encoding='utf-8-sig'
)

# 데이터 정보 출력
print_data_info(crime_df, "범죄")

In [ ]:
# 범죄 데이터 검증
validate_data(
    crime_df,
    expected_rows=25,
    required_columns=['자치구', '절도', '강도', 'CCTV효과범죄_합계']
)

# 기초 통계
print("\n📊 범죄 데이터 기초 통계:")
summary_stats = create_summary_stats(
    crime_df,
    list(CRIME_RANGE.keys()) + ['CCTV효과범죄_합계', '총_범죄']
)
summary_stats

## 3. 인구 통계 데이터 생성 및 탐색

In [ ]:
# 샘플 인구 데이터 생성
np.random.seed(RANDOM_SEED)

# 인구수 생성 (최소값 보장)
base_pop = POPULATION_CONFIG['base_population']
pop_range = POPULATION_CONFIG['population_range']
min_pop = POPULATION_CONFIG['min_population']

population_variation = np.random.randint(
    pop_range[0], pop_range[1], size=len(SEOUL_DISTRICTS)
)

population_data = {
    '자치구': SEOUL_DISTRICTS,
    '인구수': np.maximum(base_pop + population_variation, min_pop),
    '면적_km2': np.random.uniform(
        *POPULATION_CONFIG['area_range'], size=len(SEOUL_DISTRICTS)
    )
}

population_df = pd.DataFrame(population_data)

# 인구밀도 계산 (명/km²)
population_df['인구밀도'] = (
    population_df['인구수'] / population_df['면적_km2']
).round(0)

# 저장
save_csv_safely(
    population_df,
    os.path.join(DATA_PATHS['raw'], 'population_seoul_2023_sample.csv'),
    index=False,
    encoding='utf-8-sig'
)

# 데이터 정보 출력
print_data_info(population_df, "인구")

In [ ]:
# 인구 데이터 검증
validate_data(
    population_df,
    expected_rows=25,
    required_columns=['자치구', '인구수', '인구밀도']
)

# 기초 통계
print("\n📊 인구 데이터 기초 통계:")
summary_stats = create_summary_stats(
    population_df,
    ['인구수', '면적_km2', '인구밀도']
)
summary_stats

## 4. 자치구명 일치 여부 확인

In [ ]:
# 자치구명 일치 여부 확인
consistency_result = check_district_consistency(
    cctv_df, crime_df, population_df,
    district_col='자치구'
)

print(f"\n공통 자치구 수: {len(consistency_result['common_districts'])}")
print(f"전체 자치구 수: {len(consistency_result['all_districts'])}")

if consistency_result['consistent']:
    print("\n✅ 모든 데이터의 자치구명이 일치합니다. 데이터 병합 가능합니다.")
else:
    print("\n⚠️ 자치구명 불일치가 있습니다. Day 2에서 매핑 테이블 작성 필요합니다.")

## 5. 시각화: 분포 확인

In [ ]:
# CCTV 분포 시각화 (개선된 함수 사용)
data_dict = {
    '총 CCTV 대수': (cctv_df, '총_CCTV', '#1f77b4'),
    '방범용 CCTV 대수': (cctv_df, '방범용', '#2ca02c')
}

plot_multiple_distributions(
    data_dict,
    figsize=(14, 5),
    save_path=os.path.join(DATA_PATHS['figures'], 'day1_cctv_distribution.png')
)

In [ ]:
# 범죄 분포 시각화
data_dict = {
    '총 범죄 건수': (crime_df, '총_범죄', '#ff7f0e'),
    'CCTV 효과 범죄 건수': (crime_df, 'CCTV효과범죄_합계', '#d62728')
}

plot_multiple_distributions(
    data_dict,
    figsize=(14, 5),
    save_path=os.path.join(DATA_PATHS['figures'], 'day1_crime_distribution.png')
)

In [ ]:
# 인구 분포 시각화
data_dict = {
    '자치구별 인구수': (population_df, '인구수', '#9467bd'),
    '자치구별 인구밀도': (population_df, '인구밀도', '#8c564b')
}

plot_multiple_distributions(
    data_dict,
    figsize=(14, 5),
    save_path=os.path.join(DATA_PATHS['figures'], 'day1_population_distribution.png')
)

## 6. 이상치 탐지

In [ ]:
# Box plot으로 이상치 확인
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# CCTV 이상치
cctv_cols = ['총_CCTV', '방범용']
axes[0].boxplot([cctv_df[col] for col in cctv_cols], labels=cctv_cols)
axes[0].set_title('CCTV 대수 Box Plot', fontsize=14, fontweight='bold')
axes[0].set_ylabel('대수', fontsize=12)
axes[0].grid(axis='y', alpha=0.3)

# 범죄 이상치
crime_cols = ['총_범죄', 'CCTV효과범죄_합계']
axes[1].boxplot([crime_df[col] for col in crime_cols], labels=['총 범죄', 'CCTV효과범죄'])
axes[1].set_title('범죄 건수 Box Plot', fontsize=14, fontweight='bold')
axes[1].set_ylabel('건수', fontsize=12)
axes[1].grid(axis='y', alpha=0.3)

# 인구 이상치
pop_cols = ['인구수', '인구밀도']
axes[2].boxplot([population_df[col] for col in pop_cols], labels=pop_cols)
axes[2].set_title('인구 Box Plot', fontsize=14, fontweight='bold')
axes[2].set_ylabel('값', fontsize=12)
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
save_path = os.path.join(DATA_PATHS['figures'], 'day1_outliers_boxplot.png')
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"✅ 그래프 저장: {save_path}")
plt.show()

print("\n📊 이상치 분석:")
print("   - CCTV 및 범죄 데이터에서 일부 극단값 관찰")
print("   - 실제 분석 시 유동인구가 많은 지역일 가능성")
print("   - Day 2에서 자세히 검토 예정")

## 7. Day 1 요약 및 다음 단계

In [ ]:
# Day 1 요약 리포트 생성
summary = f"""
{'='*70}
Day 1 데이터 탐색 요약 (v2.0 - 개선 버전)
{'='*70}

1. 데이터 수집 완료
   ✅ CCTV 데이터: {len(cctv_df)}개 자치구, {len(cctv_df.columns)}개 컬럼
   ✅ 범죄 데이터: {len(crime_df)}개 자치구, {len(crime_df.columns)}개 컬럼
   ✅ 인구 데이터: {len(population_df)}개 자치구, {len(population_df.columns)}개 컬럼

2. 데이터 품질
   ✅ 결측치: 없음
   ✅ 자치구명 일치: {'일치' if consistency_result['consistent'] else '불일치'}
   ✅ 데이터 검증: 완료

3. CCTV 통계
   - 평균 총 CCTV 대수: {cctv_df['총_CCTV'].mean():.0f}대
   - 평균 방범용 CCTV: {cctv_df['방범용'].mean():.0f}대
   - 평균 교통단속용 CCTV: {cctv_df['교통단속용'].mean():.0f}대

4. 범죄 통계
   - 평균 총 범죄 건수: {crime_df['총_범죄'].mean():.0f}건
   - 평균 CCTV효과범죄: {crime_df['CCTV효과범죄_합계'].mean():.0f}건
   - 평균 절도: {crime_df['절도'].mean():.0f}건

5. 인구 통계
   - 평균 인구수: {population_df['인구수'].mean():.0f}명
   - 평균 인구밀도: {population_df['인구밀도'].mean():.0f}명/km²

6. 코드 개선 사항 (v2.0)
   ✅ utils 모듈 분리 (constants.py, helpers.py)
   ✅ OS별 한글 폰트 자동 설정
   ✅ 매직 넘버 제거 및 상수화
   ✅ 중복 코드 함수화
   ✅ 에러 핸들링 추가

7. 다음 단계 (Day 2 - 2025-07-05)
   - 자치구명 표준화 (필요 시)
   - CCTV 유형별 세부 집계
   - 범죄 유형별 세부 집계
   - 이상치 처리 방안 결정
   - 데이터 정제 및 저장

{'='*70}
"""

print(summary)

# 요약 파일 저장
summary_path = os.path.join(DATA_PATHS['reports'], 'day1_summary.txt')
os.makedirs(os.path.dirname(summary_path), exist_ok=True)
with open(summary_path, 'w', encoding='utf-8') as f:
    f.write(summary)
print(f"✅ 요약 파일 저장: {summary_path}")

print("\n" + "="*70)
print("🎉 Day 1 작업 완료!")
print("="*70)